In [80]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, accuracy

import warnings; warnings.simplefilter('ignore')

In [81]:
# import os
# import pandas as pd

# # 폴더 경로
# folder_path = 'watch-me-data'

# # JSON 파일 리스트 가져오기
# files = [f"{folder_path}/{file}" for file in os.listdir(folder_path) if file.endswith('.json')]

# # 데이터프레임 리스트 초기화
# dataframes = []

# # 각 파일 읽어서 리스트에 추가
# for file in files:
#     try:
#         # JSON 파일 읽기
#         data = pd.read_json(file)
#         data = data.T

#         # 데이터프레임 리스트에 추가
#         dataframes.append(data)
#         print(f"Loaded: {file}")
#     except Exception as e:
#         print(f"Error loading file {file}: {e}")

# # 모든 데이터프레임 합치기
# if dataframes:
#     combined_data = pd.concat(dataframes, ignore_index=True)

#     # 합친 데이터를 하나의 JSON 파일로 저장
#     output_file = 'combined_data.json'
#     combined_data.to_json(
#         output_file, orient='index', indent=4, force_ascii=False  # force_ascii=False로 인코딩 설정
#     )
#     print(f"All data combined and saved to: {output_file}")
# else:
#     print("No data to combine.")

In [82]:
# # 자료 합치는 용
# import json

# import os

# # 폴더 경로
# folder_path = 'watch-me-data'

# # 모든 JSON 파일 리스트 가져오기
# files = [f"{folder_path}/{file}" for file in os.listdir(folder_path) if file.endswith('.json')]

# print(files)

# # 최종 데이터를 저장할 딕셔너리
# merged_data = {}

# # 각 JSON 파일을 읽고, 데이터를 병합
# for file in files:
#     with open(file, 'r', encoding='utf-8') as f:
#         data = json.load(f)
        
#         # 데이터가 딕셔너리 형식인 경우
#         if isinstance(data, dict):
#             for tmdb_id, item in data.items():
#                 # tmdb_id를 기준으로 병합
#                 merged_data[tmdb_id] = item
#         else:
#             print(f"데이터 형식이 예상과 다릅니다. 확인이 필요합니다: {file}")

# # tmdb_id 기준으로 숫자 순으로 정렬
# sorted_data = {tmdb_id: merged_data[tmdb_id] for tmdb_id in sorted(merged_data.keys(), key=int)}

# # 결과를 result_total.json 파일로 저장 (tmdb_id 기준으로 저장됨)
# with open('watch-me-data/result_total.json', 'w', encoding='utf-8') as f:
#     json.dump(sorted_data, f, ensure_ascii=False, indent=4)

# 데이터셋 불러오기
- json 데이터셋 갖고 와서 pd.dataframe으로 변경

In [83]:
import pandas as pd
import json

# JSON 파일 경로
file_path = 'watch-me-data/combined_data.json'

# JSON 파일 읽기
with open(file_path, 'r') as file:
    md_dict = json.load(file)

# DataFrame으로 변환
md = pd.DataFrame(md_dict)

현재 행과 열이 다르게 가 있기 때문에 transpose로 바꿔준다.

In [84]:
print('////////////////////// Before ////////////////////// \n', md.columns, '\n////////////////////// Before //////////////////////')

md = md.T

print('////////////////////// After ////////////////////// \n', md.columns, '\n////////////////////// After //////////////////////')

////////////////////// Before ////////////////////// 
 Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '53426', '53427', '53428', '53429', '53430', '53431', '53432', '53433',
       '53434', '53435'],
      dtype='object', length=53436) 
////////////////////// Before //////////////////////
////////////////////// After ////////////////////// 
 Index(['kino_id', 'titleKr', 'titleEn', 'titleOri', 'synopsis', 'genres',
       'productionYear', 'posterImage', 'age_rating', 'openYear',
       'running_time', 'streaming_provider', 'country', 'releases', 'staff',
       'actor', 'tmdb_id', 'tmdb_poster_img', 'tmdb_release_date',
       'vote_count', 'vote_average', 'popularity'],
      dtype='object') 
////////////////////// After //////////////////////


In [85]:
print(md.shape)

(53436, 22)


In [86]:
md.head(3)

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,country,releases,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity
0,20016,딕,Dick,None,그저 평범하기만한 15살 소녀들 베스티와 알린. 학교 행사로 백악관 견학을 갔다가 ...,[코미디],1999,https://file.kinolights.com/original/content_p...,OVER15,1999,...,[미국],[],[앤드류 플레밍],"[미셸 윌리엄스, 커스틴 던스트]",16406,https://image.tmdb.org/t/p/original/aBG4XoK5xk...,1999-08-04,261.0,5.9,7.436
1,20018,오드 토머스,Odd Thomas,Odd Thomas,예고된 대량 살인!\r\n죽음의 냄새를 맡고 놈들이 몰려오기 시작했다!\r\n죽은 ...,"[미스터리, 스릴러, 판타지, 멜로/로맨스, 공포(호러), 코미디]",2013,https://file.kinolights.com/original/content_p...,OVER15,2014,...,"[미국, 영국]","[{'releaseDate': '2014-08-27', 'nation': {'nam...",[스티븐 소머즈],"[안톤 옐친 , 윌렘 대포, 애디슨 팀린, 애슐리 소머즈, 레오노어 바레라]",179826,https://image.tmdb.org/t/p/original/hArgTMIGfj...,2013-01-29,1326.0,6.923,21.797
2,20019,슈퍼 사이클론 스톰,Super Cyclone,None,페트로 칼 정유회사가 운영하는 윌포스-3 시추선이 굴착 작업을 진행하던 중 지각 내...,"[액션, SF]",2012,https://file.kinolights.com/original/content_p...,OVER15,2014,...,[미국],"[{'releaseDate': '2014-08-27', 'nation': {'nam...",[리즈 애덤스],[],133454,https://image.tmdb.org/t/p/original/uNlixZjRiX...,2012-09-18,39.0,4.4,4.567


In [87]:
# 인덱스를 tmdb_id 로 넣기
# 현재 인덱스를 'tmdb_id'로 설정하고 기존 컬럼 유지
md = md.set_index('tmdb_id', drop=False)

In [88]:
md.head()

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,country,releases,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity
tmdb_id,,,,,,,,,,,,,,,,,,,,,
16406,20016,딕,Dick,None,그저 평범하기만한 15살 소녀들 베스티와 알린. 학교 행사로 백악관 견학을 갔다가 ...,[코미디],1999,https://file.kinolights.com/original/content_p...,OVER15,1999,...,[미국],[],[앤드류 플레밍],"[미셸 윌리엄스, 커스틴 던스트]",16406,https://image.tmdb.org/t/p/original/aBG4XoK5xk...,1999-08-04,261.0,5.9,7.436
179826,20018,오드 토머스,Odd Thomas,Odd Thomas,예고된 대량 살인!\r\n죽음의 냄새를 맡고 놈들이 몰려오기 시작했다!\r\n죽은 ...,"[미스터리, 스릴러, 판타지, 멜로/로맨스, 공포(호러), 코미디]",2013,https://file.kinolights.com/original/content_p...,OVER15,2014,...,"[미국, 영국]","[{'releaseDate': '2014-08-27', 'nation': {'nam...",[스티븐 소머즈],"[안톤 옐친 , 윌렘 대포, 애디슨 팀린, 애슐리 소머즈, 레오노어 바레라]",179826,https://image.tmdb.org/t/p/original/hArgTMIGfj...,2013-01-29,1326.0,6.923,21.797
133454,20019,슈퍼 사이클론 스톰,Super Cyclone,None,페트로 칼 정유회사가 운영하는 윌포스-3 시추선이 굴착 작업을 진행하던 중 지각 내...,"[액션, SF]",2012,https://file.kinolights.com/original/content_p...,OVER15,2014,...,[미국],"[{'releaseDate': '2014-08-27', 'nation': {'nam...",[리즈 애덤스],[],133454,https://image.tmdb.org/t/p/original/uNlixZjRiX...,2012-09-18,39.0,4.4,4.567
254473,20020,브릭 맨션: 통제불능 범죄구역,Brick Mansions,None,"폭발까지 단 48분!\r\n오직 맨몸으로 도시를 구하라! \r\n\r\n경찰도, 군...","[액션, 범죄, 드라마]",2014,https://file.kinolights.com/original/content_p...,OVER15,2014,...,"[프랑스, 캐나다]","[{'releaseDate': '2014-08-27', 'nation': {'nam...",[카밀 들라마레],"[폴 워커, 데이비드 벨, 르자]",254473,https://image.tmdb.org/t/p/original/63pcAWCrtG...,2014-04-22,1508.0,5.9,15.495
238636,20021,더 퍼지 : 거리의 반란,The Purge: Anarchy,The Purge: Anarchy,"사이렌이 울리고, 암흑으로 뒤덮인 도시\r\n 모든 분노를 표출할 운명의 그 날이 ...","[공포(호러), 스릴러]",2014,https://file.kinolights.com/original/content_p...,ADULT,2014,...,[미국],"[{'releaseDate': '2014-08-27', 'nation': {'nam...",[제임스 드모나코],"[프랭크 그릴로, 자크 길포드, 키엘 산체즈, 마이클 K. 윌리엄즈, 채드 모건]",238636,https://image.tmdb.org/t/p/original/sbsmsts8IK...,2014-07-17,6380.0,6.64,60.962


In [89]:
print(md.loc[635302]['streaming_provider'])

tmdb_id
635302    [[넷플릭스, https://www.netflix.com/kr/title/81504...
635302    [[왓챠, https://watcha.com/contents/mWJXrZq], [티...
Name: streaming_provider, dtype: object


In [90]:
cnt = 10
for item in md['streaming_provider']:
    print(f"item: {item}")
    for i in item:
        print(f"i: {i}")
        for j in i:
            print(f"j: {j}")

    # print(item[1])

    cnt -= 1
    if cnt == 0:
        break

item: []
item: [['티빙', 'https://www.tving.com/contents/M000362498'], ['왓챠', 'https://watcha.com/contents/mWJkj8O'], ['웨이브', 'https://www.wavve.com/player/movie?movieid=MV_C901_SG0000056821']]
i: ['티빙', 'https://www.tving.com/contents/M000362498']
j: 티빙
j: https://www.tving.com/contents/M000362498
i: ['왓챠', 'https://watcha.com/contents/mWJkj8O']
j: 왓챠
j: https://watcha.com/contents/mWJkj8O
i: ['웨이브', 'https://www.wavve.com/player/movie?movieid=MV_C901_SG0000056821']
j: 웨이브
j: https://www.wavve.com/player/movie?movieid=MV_C901_SG0000056821
item: []
item: [['왓챠', 'https://watcha.com/contents/mW4L6gg'], ['U+모바일tv', 'https://motvlnk.uplus.co.kr/redirect/gateway?a_rtype=detail_page&vod_type=vod&main_run=Y&backkey_finish=N&is_splash=N&review_yn=N&contents_id=M011494294PPV00&category_id=E91VP&series_num=2974&series_category_id=']]
i: ['왓챠', 'https://watcha.com/contents/mW4L6gg']
j: 왓챠
j: https://watcha.com/contents/mW4L6gg
i: ['U+모바일tv', 'https://motvlnk.uplus.co.kr/redirect/gateway?a_rtype=de

In [91]:
print(md.head())

        kino_id           titleKr             titleEn            titleOri  \
tmdb_id                                                                     
16406     20016                 딕                Dick                None   
179826    20018            오드 토머스          Odd Thomas          Odd Thomas   
133454    20019        슈퍼 사이클론 스톰       Super Cyclone                None   
254473    20020  브릭 맨션: 통제불능 범죄구역      Brick Mansions                None   
238636    20021     더 퍼지 : 거리의 반란  The Purge: Anarchy  The Purge: Anarchy   

                                                  synopsis  \
tmdb_id                                                      
16406    그저 평범하기만한 15살 소녀들 베스티와 알린. 학교 행사로 백악관 견학을 갔다가 ...   
179826   예고된 대량 살인!\r\n죽음의 냄새를 맡고 놈들이 몰려오기 시작했다!\r\n죽은 ...   
133454   페트로 칼 정유회사가 운영하는 윌포스-3 시추선이 굴착 작업을 진행하던 중 지각 내...   
254473   폭발까지 단 48분!\r\n오직 맨몸으로 도시를 구하라! \r\n\r\n경찰도, 군...   
238636   사이렌이 울리고, 암흑으로 뒤덮인 도시\r\n 모든 분노를 표출할 운명의 그 날이 ...   

                         

In [92]:
import pandas as pd

# 필수 스트리밍 제공자 목록
required_providers = ['넷플릭스', '웨이브', '티빙', '디즈니+', '쿠팡플레이', '왓챠']

# 스트리밍 제공자 필터링
def contains_required_provider(providers, required_providers):
    # 각 provider의 이름이 필수 목록에 있는지 확인
    return any(provider[0] in required_providers for provider in providers)

# 필터링 조건 적용
md_filtered = md[md['streaming_provider'].apply(lambda x: contains_required_provider(x, required_providers))]

# 결과 출력
print(md_filtered)

        kino_id           titleKr                           titleEn  \
tmdb_id                                                               
179826    20018            오드 토머스                        Odd Thomas   
254473    20020  브릭 맨션: 통제불능 범죄구역                    Brick Mansions   
238636    20021     더 퍼지 : 거리의 반란                The Purge: Anarchy   
86001     20054                통증                              Pain   
254781    20055               플랜맨                      The Plan Man   
...         ...               ...                               ...   
890615    99961           숏버스 섬뜩행                              None   
339934    99962  스티브 맥퀸: 더 맨 앤 르망  Steve McQueen: The Man & Le Mans   
618585    99969     유령선: 죽은 자의 저주                      Blood Vessel   
712113    99970             어 굿 맨                        A Good Man   
          99975            숲을 지켜라                                     

                   titleOri  \
tmdb_id                       
179826        

In [93]:
md.shape

(53436, 22)

In [95]:
md_filtered.shape

(18172, 22)

In [97]:
md = md_filtered

## 데이터셋 각 행렬 분석

In [98]:
print(md.columns)

Index(['kino_id', 'titleKr', 'titleEn', 'titleOri', 'synopsis', 'genres',
       'productionYear', 'posterImage', 'age_rating', 'openYear',
       'running_time', 'streaming_provider', 'country', 'releases', 'staff',
       'actor', 'tmdb_id', 'tmdb_poster_img', 'tmdb_release_date',
       'vote_count', 'vote_average', 'popularity'],
      dtype='object')


In [99]:
# for feature in md.columns:
#     print(f"Feature : {feature}")
#     print(md[feature])
#     print("\n\n")

# 단순 추천 시스템
인기도나 장르 카테고리 안에서의 상위 추천.  
더 대중적이고 인기있는 영화를 유저가 더 좋아할 것이다. 를 기반 아이디어로 추천함

## 아이디어 정리
1. C : vote_averages의 평균을 구함
2. m : vote_counts의 상위 5%의 값을 구함
3. qualified : vote_counts가 상위 5%인 작품만 남긴 dataframe
4. weighted_rating : 작품 x 에 대한 vote_count와 m의 가중 평균을 구해서 'wr' 속성에 값을 넣어서 해당 값 순서대로 정렬  
    즉, 투표 수가 상위 5%인 작품 들 중에서 투표 수 대비 평점이 얼마나 좋은 지를 기준으로 정렬

In [100]:
# print('vote ::: \n', md[['vote_count', 'vote_average']].head())
# # null 값 처리 및 int로 타입 변환
# vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
# vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('float')

# print('vote ::: \n', vote_counts.head())
# print('vote ::: \n', vote_averages.head())
# # C에는 평균이 들어가 있다.
# C = vote_averages.mean()
# C

In [101]:
# # vote_counts를 내림차순으로 정렬
# vote_counts.sort_values(ascending=False)

# # quantile는 데이터를 크기대로 정렬하였을 때 분위수를 구하는 함수. quantile(0.95)는 상위 5%에 해당하는 값을 찾는 것
# m = vote_counts.quantile(0.95)
# m

In [102]:
# # 평가 수가 상위 5%인 데이터 추출
# qualified = md[(md['vote_count'] >= m) & (md['vote_count'].notnull()) & (md['vote_average'].notnull())]
# qualified['vote_count'] = qualified['vote_count'].astype('int')
# qualified['vote_average'] = qualified['vote_average'].astype('float')
# qualified.shape

In [103]:
# """
# 이 계산식은 **“많은 투표를 받은 작품일수록 해당 작품의 평점이 더 신뢰할 수 있다”**는 원칙을 따릅니다. 
# 즉, 투표 수가 적은 작품은 전체 평균 평점 C에 더 큰 영향을 받고, 투표 수가 많은 작품은 그 자체의 평균 평점 R에 더 큰 영향을 받는다는 것입니다.

# 따라서, 이 방식은 적은 투표 수로 인해 신뢰할 수 없는 평점을 보정하고, 많은 투표를 받은 작품의 평점을 우선시하여 공정하고 균형 잡힌 평점 계산을 하려고 하는 방식입니다.
# """

# def weighted_rating(x):
#     v = x['vote_count']
#     R = x['vote_average']
#     return (v/(v+m) * R) + (m/(m+v) * C)

In [104]:
# qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [105]:
# # Weighted Rating 상위 20개의 영화 
# qualified = qualified.sort_values('wr', ascending=False).head(20)

In [106]:
# qualified.head(10)

In [107]:
# # 필터링 함수 정의
# # def filter_streaming_providers(df, providers):
# #     # 복사본 생성
# #     filtered_df = df.copy()

# #     # 각 행의 streaming_provider에서 원하는 제공자만 필터링
# #     def filter_providers(providers_list):
# #         return [
# #             provider for provider in providers_list
# #             if any(desired in provider[0] for desired in providers)
# #         ]

# #     # 'streaming_provider' 열 필터링
# #     filtered_df["streaming_provider"] = filtered_df["streaming_provider"].apply(filter_providers)
# #     # 원하는 제공자가 포함된 행만 유지
# #     return filtered_df[filtered_df["streaming_provider"].map(len) > 0]
# def filter_streaming_providers(df, providers):
#     # 복사본 생성
#     filtered_df = df.copy()

#     # 각 행의 streaming_provider에서 원하는 제공자만 필터링
#     def filter_providers(providers_list):
#         return [
#             provider for provider in providers_list
#             if any(desired in provider[0] for desired in providers)
#         ]

#     # 'streaming_provider' 열 필터링
#     filtered_df["streaming_provider"] = filtered_df["streaming_provider"].apply(filter_providers)
#     # 원하는 제공자가 포함된 행만 유지
#     return filtered_df[filtered_df["streaming_provider"].map(len) > 0]

In [108]:
# md['streaming_provider'].head()

In [109]:
# # 원하는 provider 목록
# desired_providers = ["넷플릭스", "티빙"]

# # 필터링 실행
# filtered_data = filter_streaming_providers(md, desired_providers)

# # 결과 출력
# filtered_data['streaming_provider'].head()

In [110]:
# def filter_genre(df, genres):
#     # 여러 장르 중 하나라도 포함된 행 필터링
#     qualified_genre = df.copy()
#     qualified_genre = qualified_genre[qualified_genre['genres'].apply(
#         lambda x: any(genre in x for genre in genres)
#     )]
#     return qualified_genre

In [111]:
# qualified_genre = filter_genre(qualified, ['액션', '로맨스'])
# qualified_genre.head(10)

 # 콘텐츠 기반 추천
**사용자가 이전에 좋아했던 콘텐츠의 특징을 분석하고, 이와 유사한 속성을 가진 콘텐츠 추천**

1. description(tag나 설명)을 갖고오기
2. BOW(Bag Of Words) 방식과 TF-IDF로 문서를 벡터화 시킴 (단어가 나온 횟수)
3. 코사인 유사도로 두 영화 사이의 유사도를 계산  
**여기에 메타 데이터를 추가**  

4. crew, cast에서 갑독과 배우진들 추출  
5. 키워드 추출  
6. Count Vectorizer를 마찬가지로 사용하여 코사인 유사도 계산  

**여기에 인기도와 평점을 추가**  

7. 인기도와 평점을 갖고와서 평이 좋지 않은 나쁜 영화를 제거해줌


## Synopsis를 BOW로 벡터화

1. 설명을 토큰화
2. 토큰화 한 내용을 tfidf로 벡터화 (갯수가 몇 개 나왔는 지)

In [112]:
md['synopsis'].head()

tmdb_id
179826    예고된 대량 살인!\r\n죽음의 냄새를 맡고 놈들이 몰려오기 시작했다!\r\n죽은 ...
254473    폭발까지 단 48분!\r\n오직 맨몸으로 도시를 구하라! \r\n\r\n경찰도, 군...
238636    사이렌이 울리고, 암흑으로 뒤덮인 도시\r\n 모든 분노를 표출할 운명의 그 날이 ...
86001     강풀 원안 곽경택 감독의 슬프지만 아름다운 사랑\r\n 어릴 적 자신의 실수 때문에...
254781    6:00 기상, 밤새 흐트러진 침구 다림질. 6:35 샤워, 드라이기로 욕실 물기 ...
Name: synopsis, dtype: object

In [113]:
# 모든 컬럼에 대해 각 항목이 리스트일 경우 텍스트 형태로 이어붙이기
md = md.applymap(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else x)

# 결과 출력 (변경된 데이터)
md.head()

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,country,releases,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity
tmdb_id,,,,,,,,,,,,,,,,,,,,,
179826,20018,오드 토머스,Odd Thomas,Odd Thomas,예고된 대량 살인!\r\n죽음의 냄새를 맡고 놈들이 몰려오기 시작했다!\r\n죽은 ...,미스터리 스릴러 판타지 멜로/로맨스 공포(호러) 코미디,2013,https://file.kinolights.com/original/content_p...,OVER15,2014,...,미국 영국,"{'releaseDate': '2014-08-27', 'nation': {'name...",스티븐 소머즈,안톤 옐친 윌렘 대포 애디슨 팀린 애슐리 소머즈 레오노어 바레라,179826,https://image.tmdb.org/t/p/original/hArgTMIGfj...,2013-01-29,1326.0,6.923,21.797
254473,20020,브릭 맨션: 통제불능 범죄구역,Brick Mansions,None,"폭발까지 단 48분!\r\n오직 맨몸으로 도시를 구하라! \r\n\r\n경찰도, 군...",액션 범죄 드라마,2014,https://file.kinolights.com/original/content_p...,OVER15,2014,...,프랑스 캐나다,"{'releaseDate': '2014-08-27', 'nation': {'name...",카밀 들라마레,폴 워커 데이비드 벨 르자,254473,https://image.tmdb.org/t/p/original/63pcAWCrtG...,2014-04-22,1508.0,5.900,15.495
238636,20021,더 퍼지 : 거리의 반란,The Purge: Anarchy,The Purge: Anarchy,"사이렌이 울리고, 암흑으로 뒤덮인 도시\r\n 모든 분노를 표출할 운명의 그 날이 ...",공포(호러) 스릴러,2014,https://file.kinolights.com/original/content_p...,ADULT,2014,...,미국,"{'releaseDate': '2014-08-27', 'nation': {'name...",제임스 드모나코,프랭크 그릴로 자크 길포드 키엘 산체즈 마이클 K. 윌리엄즈 채드 모건,238636,https://image.tmdb.org/t/p/original/sbsmsts8IK...,2014-07-17,6380.0,6.640,60.962
86001,20054,통증,Pain,통증,강풀 원안 곽경택 감독의 슬프지만 아름다운 사랑\r\n 어릴 적 자신의 실수 때문에...,드라마 멜로/로맨스,2011,https://file.kinolights.com/original/content_p...,OVER15,2011,...,한국,"{'releaseDate': '2011-09-07', 'nation': {'name...",곽경택,권상우 정려원 마동석,86001,https://image.tmdb.org/t/p/original/wZ4CTkFITa...,2011-09-07,28.0,6.500,2.902
254781,20055,플랜맨,The Plan Man,None,"6:00 기상, 밤새 흐트러진 침구 다림질. 6:35 샤워, 드라이기로 욕실 물기 ...",드라마 코미디,2013,https://file.kinolights.com/original/content_p...,OVER15,2014,...,한국,"{'releaseDate': '2014-01-09', 'nation': {'name...",성시흡,정재영 한지민,254781,https://image.tmdb.org/t/p/original/8N7ySkDva4...,2014-01-09,29.0,6.800,4.332


In [114]:
# 한글 tokenizer 만들기
from konlpy.tag import Okt  

okt = Okt()

# 형태소 분석 함수
def korean_tokenizer(text):
    return okt.morphs(text)  # 형태소 단위로 나눔

In [28]:
# # 토큰화가 잘 되는 지 확인
# cnt = 5
# for text in md['synopsis']:
#     print(text)
#     print(okt.morphs(text))
#     print()
#     cnt -= 1
#     if cnt == 0:
#         break

In [29]:
# TfidfVectorizer 사용 (tokenizer에 형태소 분석기 함수 전달)
tf = TfidfVectorizer(tokenizer=korean_tokenizer)

# TF-IDF 매트릭스 생성
tfidf_matrix = tf.fit_transform(md['synopsis'])

# 결과 출력
print("특징어들:", tf.get_feature_names_out())
print("TF-IDF 행렬:")
print(tfidf_matrix.toarray())

특징어들: ['\x08' '\t\r\n' '\t\r\n \r\n ' ... '］' 'ﾠ' '🌟']
TF-IDF 행렬:
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [30]:
tfidf_matrix.shape

(53436, 115846)

In [31]:
# 각 문서에서 중요한 단어들을 추출 (상위 5개 키워드 선택)
keywords = []
for row in tfidf_matrix:
    # 각 영화 설명에서 상위 5개 키워드 추출
    indices = row.toarray().flatten().argsort()[-5:][::-1]  # TF-IDF 값이 높은 상위 5개 단어의 인덱스
    top_keywords = [tf.get_feature_names_out()[i] for i in indices]
    keywords.append(top_keywords)

# 결과를 md['keyword']에 저장
md['keyword'] = keywords

# 결과 출력
print(md[['synopsis', 'keyword']])

KeyboardInterrupt: 

In [ ]:
tfidf_matrix.shape

(4495, 35891)

## 코사인 유사도

코사인 유사도(Cosine Similarity)을 사용하여 두 영화 사이의 유사성을 나타내는 숫자 수량을 계산할 것입니다. 수학적으로 다음과 같이 정의됩니다.

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

TF-IDF 벡터 라이저를 사용 했으므로 Dot Product를 계산하면 코사인 유사도 점수를 직접 얻을 수 있습니다. 따라서 cosine_similarities 대신 sklearn의 **linear_kernel**을 사용하는 것이 훨씬 빠릅니다.


In [ ]:
# # ToDo: load하는 주소들 변경
# cosine_sim_description = np.load(
#     'final/data/cosine_sim_description.npy')
# # ToDo: load하는 주소들 변경
# cosine_sim_metadata = np.load(
#     'final/data/cosine_sim_metadata.npy')

In [ ]:
# cosine_sim_description

array([[1.        , 0.04132147, 0.06898292, ..., 0.06572524, 0.05839973,
        0.02628727],
       [0.04132147, 1.        , 0.03791466, ..., 0.05584348, 0.05103352,
        0.01094246],
       [0.06898292, 0.03791466, 1.        , ..., 0.07949401, 0.04912776,
        0.01686608],
       ...,
       [0.06572524, 0.05584348, 0.07949401, ..., 1.        , 0.0832154 ,
        0.02024433],
       [0.05839973, 0.05103352, 0.04912776, ..., 0.0832154 , 1.        ,
        0.0368333 ],
       [0.02628727, 0.01094246, 0.01686608, ..., 0.02024433, 0.0368333 ,
        1.        ]])

In [ ]:
# # linear_kernel는 두 벡터의 dot product 이다.
# cosine_sim_description = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
# print(cosine_sim_description.shape)

(4495, 4495)


In [ ]:
# # titleKr이 '과속스캔들'인 행 찾기
# result = md[md['titleKr'] == "과속스캔들"]

# result.columns

Index(['kino_id', 'titleKr', 'titleEn', 'titleOri', 'synopsis', 'genres',
       'productionYear', 'posterImage', 'age_rating', 'openYear',
       'running_time', 'streaming_provider', 'country', 'releases', 'staff',
       'actor', 'tmdb_id', 'tmdb_poster_img', 'tmdb_release_date',
       'vote_count', 'vote_average', 'popularity', 'soup', 'soup_search'],
      dtype='object')

In [ ]:
# tmp_md = md.copy()
# tmp_md = tmp_md.reset_index()
# # tmp_md['tmdb_id'] = tmp_md['tmdb_id'].astype('int')
# titles = tmp_md['titleKr']
# # indices = pd.Series(tmp_md.index, index=tmp_md['titleKr'])
# # titleKr을 인덱스로 하고 tmdb_id를 값으로 매핑하는 Series 생성
# # titleMap = pd.Series(md['tmdb_id'].values, index=tmp_md.index)

# # titleKr에 해당하는 tmdb_id 반환
# # title = "과속스캔들"  # 예시 제목
# # tmdb_id = indices.get(title)  # titleKr을 넣어서 tmdb_id 가져오기
# # print(f"TMDB ID for '{title}': {tmdb_id}")

# # titles: id → title 로 매핑
# # indices: title → id 로 매핑
# print(titles.head(), indices.head())

0    스타워즈 에피소드 4 - 새로운 희망
1                 니모를 찾아서
2     캐리비안의 해적 - 블랙 펄의 저주
3                 킬 빌: 1부
4                 지옥의 묵시록
Name: titleKr, dtype: object titleKr
스타워즈 에피소드 4 - 새로운 희망    0
니모를 찾아서                 1
캐리비안의 해적 - 블랙 펄의 저주     2
킬 빌: 1부                 3
지옥의 묵시록                 4
dtype: int64


In [ ]:
# tmp_md를 복사하고 reset_index
tmp_md = md.copy()
tmp_md = tmp_md.reset_index()

# tmdbId_to_matIdx: tmdb_id를 기준으로 index를 매핑
tmdbId_to_matIdx = pd.Series(tmp_md.index.values, index=tmp_md['tmdb_id']).to_dict()

# matIdx_to_tmdbId: index를 기준으로 tmdb_id를 매핑
matIdx_to_tmdbId = pd.Series(tmp_md['tmdb_id'].values, index=tmp_md.index).to_dict()

# 결과 확인
print("tmdbId_to_matIdx:", tmdbId_to_matIdx)
print("matIdx_to_tmdbId:", matIdx_to_tmdbId)

tmdbId_to_matIdx: {11: 0, 12: 1, 22: 2, 24: 3, 28: 4, 38: 5, 58: 6, 63: 7, 74: 8, 76: 9, 77: 10, 79: 11, 81: 12, 87: 13, 89: 14, 98: 15, 99: 16, 101: 17, 103: 18, 106: 19, 108: 20, 113: 21, 114: 22, 120: 23, 121: 24, 122: 25, 128: 26, 129: 27, 137: 28, 140: 29, 142: 30, 146: 31, 149: 32, 158: 33, 165: 34, 167: 35, 194: 36, 196: 37, 214: 38, 217: 39, 223: 40, 226: 41, 238: 42, 240: 43, 241: 44, 242: 45, 246: 46, 262: 47, 266: 48, 272: 49, 280: 50, 296: 51, 300: 52, 329: 53, 330: 54, 331: 55, 336: 56, 345: 57, 346: 58, 348: 59, 389: 60, 390: 61, 393: 62, 395: 63, 402: 64, 404: 65, 409: 66, 411: 67, 424: 68, 426: 69, 445: 70, 490: 71, 499: 72, 509: 73, 524: 74, 539: 75, 557: 76, 558: 77, 562: 78, 563: 79, 564: 80, 577: 81, 581: 82, 582: 83, 584: 84, 585: 85, 591: 86, 594: 87, 598: 88, 601: 89, 603: 90, 604: 91, 605: 92, 607: 93, 608: 94, 612: 95, 613: 96, 615: 97, 635: 98, 652: 99, 661: 100, 666: 101, 672: 102, 673: 103, 674: 104, 675: 105, 679: 106, 694: 107, 695: 108, 713: 109, 744: 110

In [ ]:
# titleMap[indices['과속스캔들']]

18384

### .npy 방식으로 코사인 유사도 행렬 저장


In [ ]:
# 저장

import numpy as np

# NumPy 배열을 바로 저장
# np.save('cosine_sim_description.npy', cosine_sim_description)
# np.save('final/data/cosine_sim_metadata.npy', cosine_sim_metadata)

In [ ]:
# import numpy as np

# # NumPy 배열을 바로 로드
# tmp_cosine_sim_description = np.load('cosine_sim_description.npy')
# tmp_cosine_sim_metadata = np.load('cosine_sim_metadata.npy')

In [ ]:
import pickle

# Series 저장
with open('final/data/tmdbId_to_matIdx.pkl', 'wb') as f:
    pickle.dump(tmdbId_to_matIdx, f)

In [ ]:
import pickle

# Series 저장
with open('final/data/matIdx_to_tmdbId.pkl', 'wb') as f:
    pickle.dump(matIdx_to_tmdbId, f)

In [ ]:
md.to_pickle('keyword_md.pkl')

In [ ]:
# import pickle

# with open('final/data/indices.pkl', 'rb') as f:
#     indices = pickle.load(f)

# with open('final/data/titleMap.pkl', 'rb') as f:
#     titleMap = pickle.load(f)

# cosine_sim_metadata = np.load('final/data/cosine_sim_metadata.npy')

In [ ]:
# """
# 콘텐츠 기반 추천: 특정 작품과 유사한 작품을 추천
# 사용자의 시청 기록들 및 평가들을 보고 해당 작품과 유사한 작품 여러 개를 추천해 줄 수 있음

# 필요한 리스트
# 1. indices: id → title 로의 매핑
# 2. cosine_sim_description: tf-idf의 코사인 유사도 행렬
# """
# def get_recommendations(title):
#     idx = indices[title]
#     sim_scores = list(enumerate(cosine_sim_description[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31] # 유사한 상위 30개만 선택하겠다.
#     # sim_scores = sim_scores[0:30] # 만약 자기 자신 포함하고 싶으면 이거하면 됨
#     movie_indices = [i[0] for i in sim_scores]
#     return titles.iloc[movie_indices]

In [ ]:
# get_recommendations('어게인 1997')

2317            안녕, 나의 소녀
1559            센트럴 인텔리전스
1606                   동승
4225          생각 그런거 없는데요
2465         어느 날 인생이 엉켰다
3430              스퀘어드 러브
3659    용쟁호투: 나는 드래곤이 아니다
2586               배반의 장미
770                     M
1441         미안해 사랑이라 불러서
1279              그레이트 뷰티
4366                    씬
3652            견자단의 용호무사
3724            줄리아의 인생극장
1881               팬텀 스레드
3669                   실종
1671               미쓰 와이프
1539         돈키호테를 죽인 사나이
272             과거가 없는 남자
2702                   우상
1983             아이 리멤버 유
2696     지푸라기라도 잡고 싶은 짐승들
1104       플레이스 비욘드 더 파인즈
635              하나 그리고 둘
133          잃어버린 아이들의 도시
2134          변함없는 자들의 마을
4065                 여도둑들
3152            마음에 부는 바람
3986                  신도들
2648               루시드 드림
Name: titleKr, dtype: object

## 메타데이터 기반 추천 시스템
감독과 배우진들과 키워드를 기반으로 추천

Count Vectorizer를 이용하여 plot에서 한 것처럼, 카운트 매트릭스를 만들자.  

In [129]:
# pmd에 md를 복사해서 넣는다.
pmd = md.copy()

In [149]:
# 감독의 가중치를 3배로 넣기 위해 가중치를 3번 넣는다.
pmd['staff'] = pmd['staff'].apply(lambda x: [x,x,x,x,x,x,x,x,x])
pmd['genres'] = pmd['genres'].apply(lambda x: [x,x,x,x])
pmd['actor'] = pmd['genres'].apply(lambda x: [x,x,x,x,x,x,x,x,x,x])

In [150]:
pmd['genres'].head()

11    [[[SF, 어드벤처(모험), 액션, 판타지], [SF, 어드벤처(모험), 액션, ...
12    [[[애니메이션, 어드벤처(모험), 가족, 코미디], [애니메이션, 어드벤처(모험)...
22    [[[어드벤처(모험), 액션, 코미디, 판타지], [어드벤처(모험), 액션, 코미디...
24    [[[액션, 스릴러, 범죄, 드라마], [액션, 스릴러, 범죄, 드라마], [액션,...
28    [[[전쟁, 드라마, 액션], [전쟁, 드라마, 액션], [전쟁, 드라마, 액션],...
Name: genres, dtype: object

In [151]:
# 중첩된 리스트를 재귀적으로 풀어주는 함수
def flatten_list(x):
    if isinstance(x, list):
        # 리스트가 있을 경우 각 항목을 재귀적으로 처리하여 풀어줌
        return [item for sublist in x for item in flatten_list(sublist)]
    else:
        # 리스트가 아니면 해당 항목을 그대로 반환
        return [x]

# 중첩된 리스트를 풀고, 공백으로 이어붙이는 함수
def flatten_and_join(x):
    flattened = flatten_list(x)
    return ' '.join(map(str, flattened))

# 각 열에 대해 중첩된 리스트를 풀어서 합침
pmd['soup'] = pmd['staff'].apply(flatten_and_join) + ' ' + \
              pmd['actor'].apply(flatten_and_join) + ' ' + \
              pmd['genres'].apply(flatten_and_join) + ' ' + \
              pmd['keyword'].apply(flatten_and_join)

# 확인
print(pmd['soup'].head())

11    조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루...
12    앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠...
22    고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스...
24    쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠...
28    프란시스 포드 코폴라 프란시스 포드 코폴라 프란시스 포드 코폴라 프란시스 포드 코폴...
Name: soup, dtype: object


In [152]:
pmd['soup'].head()

11    조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루카스 조지 루...
12    앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠...
22    고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스키 고어 버빈스...
24    쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠...
28    프란시스 포드 코폴라 프란시스 포드 코폴라 프란시스 포드 코폴라 프란시스 포드 코폴...
Name: soup, dtype: object

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# # 한국어 불용어 리스트 (예시)
# stop_words_ko = ['이', '그', '저', '것', '들', '과', '에', '의', '는', '가', '이랑', '하고', '도', '에서', '을', '를']

# # CountVectorizer 생성 시 stop_words 매개변수에 리스트 전달
# count = CountVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words_ko)

# # 'soup' 열에 대해 변환 수행
# count_matrix = count.fit_transform(pmd['soup'])

In [172]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 한국어 불용어 리스트 (예시)
stop_words_ko = ['이', '그', '저', '것', '들', '과', '에', '의', '는', '가', '이랑', '하고', '도', '에서', '을', '를']

# TfidfVectorizer 생성 시 stop_words 매개변수에 리스트 전달
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words=stop_words_ko)

# 'soup' 열에 대해 변환 수행
tfidf_matrix = tfidf.fit_transform(pmd['soup'])

In [ ]:
# cosine_sim을 count_matrix에 대해서 업데이트
# 현재는 tf-idf 방식과 동일하게 들어가 있는데
cosine_sim_metadata = cosine_similarity(tfidf_matrix, tfidf_matrix)
# cosine_sim_metadata0 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# 저장

import numpy as np

# NumPy 배열을 바로 저장
# np.save('cosine_sim_description.npy', cosine_sim_description)
np.save('final/data/cosine_sim_metadata.npy', cosine_sim_metadata)


In [ ]:
import numpy as np

# cosine_sim_metadata를 압축된 npz 파일로 저장
np.savez_compressed('final/data/cosine_sim_metadata.npz', cosine_sim_metadata=cosine_sim_metadata)

In [ ]:
# """
# 단순 가중 합산 평균
# """
# def get_recommendations_of_contents(title, weight_description=0, weight_metadata=1):
#     idx = indices[title]
    
#     # 두 유사도 행렬을 결합
#     # sim_scores_description = list(enumerate(cosine_sim_description[idx]))
#     sim_scores = list(enumerate(cosine_sim_metadata[idx]))
    
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]  # 상위 30개
#     movie_indices = [i[0] for i in sim_scores]


#     movies = titleMap[movie_indices]
#     result_idx = [str(i) for i in movies]
#     return md.loc[result_idx][:10]
    
#     return titles.iloc[movie_indices]



# """
# 유사도 합산 후 평균 버전
# """

# # def get_recommendations_of_contents(title):
# #     idx = indices[title]
    
# #     # 두 유사도 행렬을 결합하여 평균을 구함
# #     sim_scores_description = list(enumerate(cosine_sim_description[idx]))
# #     sim_scores_metadata = list(enumerate(cosine_sim_metadata[idx]))
    
# #     # 두 유사도 값의 평균을 취함
# #     sim_scores = [
# #         (i[0], (i[1] + j[1]) / 2)
# #         for i, j in zip(sim_scores_description, sim_scores_metadata)
# #     ]
    
# #     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
# #     sim_scores = sim_scores[:30]  # 상위 30개
# #     movie_indices = [i[0] for i in sim_scores]
    
# #     return titles.iloc[movie_indices]

'\n유사도 합산 후 평균 버전\n'

In [ ]:
# def recommend_hybrid_one(user_id, title, ott_list):
#         """
#         사용자에 알맞는 맞춤 추천을 제공

#         input: user_id, title
#         """

#         # md = self.get_md_filtered_by_ott_list(self.md_original, ott_list)
#         new_md = md.copy()

#         # 영화 제목을 통해 인덱스 가져오기
#         idx = indices[title]
    

#         # 영화 설명 유사도 (TF-IDF)와 메타데이터 유사도 (감독, 배우, 장르) 가져오기
#         sim_scores = list(enumerate(cosine_sim_metadata[int(idx)]))

#         # md에서 유효한 인덱스를 가져오기
#         valid_indices = new_md.index.tolist()

#         # sim_scores에서 md에 포함된 인덱스만 남기도록 필터링
#         # sim_scores_filtered = [score for score in sim_scores if score[0] in valid_indices]

#         # 유사도 순으로 정렬 (가장 유사한 것부터)
#         sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#         # 상위 25개 영화 선택 (자기 자신 제외)
#         sim_scores = sim_scores[1:26]
#         movie_indices = [i[0] for i in sim_scores]

#         # 영화 데이터 가져오기 (제목, 투표수, 평균 평점, 년도, id)
#         movies = titleMap[movie_indices]
#         result_idx = [str(i) for i in movies]
#         return new_md.loc[result_idx][:5]

#         # 최소 vote_count를 기준으로 필터링 (예: 100 이상)
#         min_vote_count = 1
#         movies = movies[movies['vote_count'] >= min_vote_count]

#         # SVD를 통해 예측된 평점 계산
#         # movies['est'] = movies['tmdb_id'].apply(lambda x: self.svd.predict(user_id, x).est)

#         # # 예측된 평점 기준으로 상위 10개 영화 추천
#         # movies = movies.sort_values('est', ascending=False)

#         return movies[:10]

In [ ]:
# recommend_hybrid_one("se1", "범죄도시 2", [])

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,country,releases,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity
955555,110565,범죄도시 3,The Roundup : No Way Out,범죄도시 3,"베트남 납치 살해범 검거 후 7년 뒤, 마석도는 새로운 팀원들과 함께 살인사건을 조...","[액션, 범죄]",2022,https://file.kinolights.com/original/content_p...,OVER15,2023,...,[한국],"[{'releaseDate': '2023-05-31', 'nation': {'nam...",[이상용],"[마동석, 이준혁, 아오키 무네타카]",955555,https://image.tmdb.org/t/p/original/kPtNokhBtg...,2023-05-31,427.0,7.2,76.339
58414,48032,테러리스트,A True Mob Story,龍在江湖,청대(유덕화 분)는 조직 폭력배의 말단. 지금은 보잘 것 없지만 언젠가는 조직의 보...,"[액션, 범죄]",1997,https://file.kinolights.com/original/content_p...,OVER15,1999,...,"[중국, 홍콩]","[{'releaseDate': '1999-01-01', 'nation': {'nam...",[왕정],[유덕화],58414,https://image.tmdb.org/t/p/original/fRUz2HAs5v...,1998-07-01,15.0,6.8,6.21
110410,17777,마약전쟁,Drug War,毒戰,중국 본토의 수사관 장(손홍뢰)은 어느 날 홍콩 마약 조직의 보스 차이(고천락)를 ...,"[액션, 범죄]",2013,https://file.kinolights.com/original/content_p...,ADULT,2014,...,"[홍콩, 중국]","[{'releaseDate': '2014-02-13', 'nation': {'nam...",[두기봉],"[고천락 , 임가동, 순홍레이, 황혁, 협선]",110410,https://image.tmdb.org/t/p/original/dgR7BgjtX7...,2012-11-15,268.0,6.9,10.933
513692,99887,레이징 파이어,Raging Fire,怒火,강력 범죄 수사대에서 함께 믿고 일하던 베테랑 경찰 장충방과 그의 후배 추강아오. ...,"[액션, 범죄]",2021,https://file.kinolights.com/original/content_p...,OVER15,2022,...,"[홍콩, 중국]","[{'releaseDate': '2022-01-20', 'nation': {'nam...",[진목승],"[견자단, 사정봉]",513692,https://image.tmdb.org/t/p/original/xYE8E7OF9w...,2021-07-28,216.0,6.928,20.559
891699,132255,사일런트 나잇,Silent Night,Silent Night,"갱단의 총격전에서 아들과 목소리까지 잃은 남자. 삶을 포기하려는 순간, 가장 죽기에...","[액션, 범죄]",2023,https://file.kinolights.com/original/content_p...,ADULT,2023,...,[미국],"[{'releaseDate': '2024-07-17', 'nation': {'nam...",[오우삼],"[조엘 킨나만, 키드 커디, 카타리나 산디노 모레노, 해롤드 토레스]",891699,https://image.tmdb.org/t/p/original/n42g6abIQa...,2023-11-30,735.0,6.253,45.448


In [ ]:
# get_recommendations_of_contents('범죄도시 2')

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,country,releases,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity
955555,110565,범죄도시 3,The Roundup : No Way Out,범죄도시 3,"베트남 납치 살해범 검거 후 7년 뒤, 마석도는 새로운 팀원들과 함께 살인사건을 조...","[액션, 범죄]",2022,https://file.kinolights.com/original/content_p...,OVER15,2023,...,[한국],"[{'releaseDate': '2023-05-31', 'nation': {'nam...",[이상용],"[마동석, 이준혁, 아오키 무네타카]",955555,https://image.tmdb.org/t/p/original/kPtNokhBtg...,2023-05-31,427.0,7.2,76.339
58414,48032,테러리스트,A True Mob Story,龍在江湖,청대(유덕화 분)는 조직 폭력배의 말단. 지금은 보잘 것 없지만 언젠가는 조직의 보...,"[액션, 범죄]",1997,https://file.kinolights.com/original/content_p...,OVER15,1999,...,"[중국, 홍콩]","[{'releaseDate': '1999-01-01', 'nation': {'nam...",[왕정],[유덕화],58414,https://image.tmdb.org/t/p/original/fRUz2HAs5v...,1998-07-01,15.0,6.8,6.21
110410,17777,마약전쟁,Drug War,毒戰,중국 본토의 수사관 장(손홍뢰)은 어느 날 홍콩 마약 조직의 보스 차이(고천락)를 ...,"[액션, 범죄]",2013,https://file.kinolights.com/original/content_p...,ADULT,2014,...,"[홍콩, 중국]","[{'releaseDate': '2014-02-13', 'nation': {'nam...",[두기봉],"[고천락 , 임가동, 순홍레이, 황혁, 협선]",110410,https://image.tmdb.org/t/p/original/dgR7BgjtX7...,2012-11-15,268.0,6.9,10.933
513692,99887,레이징 파이어,Raging Fire,怒火,강력 범죄 수사대에서 함께 믿고 일하던 베테랑 경찰 장충방과 그의 후배 추강아오. ...,"[액션, 범죄]",2021,https://file.kinolights.com/original/content_p...,OVER15,2022,...,"[홍콩, 중국]","[{'releaseDate': '2022-01-20', 'nation': {'nam...",[진목승],"[견자단, 사정봉]",513692,https://image.tmdb.org/t/p/original/xYE8E7OF9w...,2021-07-28,216.0,6.928,20.559
891699,132255,사일런트 나잇,Silent Night,Silent Night,"갱단의 총격전에서 아들과 목소리까지 잃은 남자. 삶을 포기하려는 순간, 가장 죽기에...","[액션, 범죄]",2023,https://file.kinolights.com/original/content_p...,ADULT,2023,...,[미국],"[{'releaseDate': '2024-07-17', 'nation': {'nam...",[오우삼],"[조엘 킨나만, 키드 커디, 카타리나 산디노 모레노, 해롤드 토레스]",891699,https://image.tmdb.org/t/p/original/n42g6abIQa...,2023-11-30,735.0,6.253,45.448
138529,3949,냉혈저격수,Tiger Cage 3,冷面狙擊手,경찰인 제임스와 존은 대부호 이소방을 수사하며 도청하던 중 기밀 주식 정보를 알게 ...,"[범죄, 액션]",1991,https://file.kinolights.com/original/content_p...,OVER15,2018,...,[홍콩],"[{'releaseDate': '2018-02-01', 'nation': {'nam...",[원화평],"[장국량, 왕민덕, 장민, 황금강, 장오랑, 후펑, 유순, 사미의]",138529,https://image.tmdb.org/t/p/original/kVeIophS1d...,1991-11-14,10.0,5.7,6.044
814785,125717,지스톰: 범죄와의전쟁,G Storm,G風暴,홍콩의 부정부패를 척결하는 비밀부서 ICAC의 반장 육지렴과 주임 정덕명은 한 사건...,"[액션, 범죄]",2021,https://file.kinolights.com/original/content_p...,OVER15,2021,...,[홍콩],"[{'releaseDate': '2021-12-31', 'nation': {'nam...",[데이빗 램],"[고천락 , 정가영, 선훤]",814785,https://image.tmdb.org/t/p/original/ik621LxS01...,2021-12-31,21.0,5.0,4.984
58412,24630,노호광,Tiger Cage 2,洗黑錢,"홍콩의 변호사들인 오대유, 소걸, 조위생은 두목인 조숙 밑에서 미국에서 홍콩으로 돈...","[액션, 범죄]",1992,https://file.kinolights.com/original/content_p...,OVER15,1992,...,[홍콩],"[{'releaseDate': '1992-12-05', 'nation': {'nam...",[원화평],"[견자단, 관지림, 오대유, 위룡, 정유령, 양리칭, 나열]",58412,https://image.tmdb.org/t/p/original/44l2hi5vWr...,1990-08-11,30.0,6.95,10.271
58409,24321,특경도룡,Tiger Cage,特警屠龍,"말보다는 주먹, 법보다는 총!\r\n음모와 배신으로 물든 무법도시!\r\n홍콩 경시...","[액션, 범죄]",1988,https://file.kinolights.com/original/content_p...,OVER15,1989,...,[홍콩],"[{'releaseDate': '1989-03-01', 'nation': {'nam...",[원화평],"[견자단, 장학우, 임달화, 정유령, 원비샤]",58409,https://image.tmdb.org/t/p/original/zTOuYvMcz5...,1988-07-28,35.0,6.4,7.15
42120,14878,용호풍운,City On Fire,龍虎風雲,"비밀경찰의 희생으로 경찰이 딜레마에 빠진 가운데, 사건을 맡은 수사관 라우는 비밀경...","[범죄, 액션]",1987,https://file.kinolights.com/original/content_p...,OVER15,1987,...,[홍콩],"[{'releaseDate': '1988-12-16', 'nation': {'nam...",[임영동],"[주윤발 , 이수현 ]",42120,https://image.tmdb.org/t/p/original/lYRc5LOZK6...,1987-02-13,140.0,7.0,13.687


## 인기도와 평점

인기도나 평점이 너무 낮은 애들은 빼준다

In [ ]:
# type(pmd)

pandas.core.frame.DataFrame

In [ ]:
# pmd.to_pickle('pmd.pkl')

In [ ]:
# # DataFrame 불러오기
# tmp_pmd = pd.read_pickle('pmd.pkl')

In [ ]:
# """
# 기존 추천 함수에서 평점이 너무 낮은 애들은 빼주었다.

# 필요한 것
# 1. pmd: soup = staff + actor + genre 가 있는 md
# 2. indices: title → id   
# 3. cosine_sim_description: tf-idf 방식의 코사인 유사도 행렬
# 4. cosine_sim_metadat: metadata(staff, actor, genre)의 counter vectorizer의 코사인 유사도

# """

# def improved_recommendations(title, weight_description=0.2, weight_metadata=0.8):
#     print(f"추천할 영화: {title}")
#     idx = indices[title]
#     print(md.iloc[idx]['synopsis'])
#     print(f"영화 인덱스: {idx}")
    
#     # 영화 설명 유사도 (TF-IDF)
#     sim_scores_description = list(enumerate(cosine_sim_description[idx]))
#     # 메타데이터 유사도 (감독, 배우, 장르)
#     sim_scores_metadata = list(enumerate(cosine_sim_metadata[idx]))
    
#     # 두 유사도 행렬 결합 (가중합)
#     sim_scores = [
#         (i[0], weight_description * i[1] + weight_metadata * j[1])
#         for i, j in zip(sim_scores_description, sim_scores_metadata)
#     ]
    
#     # 유사도 순으로 정렬 (가장 유사한 것부터)
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
#     # 상위 25개 영화 선택 (자기 자신 제외)
#     sim_scores = sim_scores[1:26]
#     movie_indices = [i[0] for i in sim_scores]
#     print(f"추천할 영화들 인덱스: {movie_indices}")
    
#     # 영화 데이터 가져오기 (이름, 투표수, 평균 평점, 년도)
#     # movies = tmp_pmd.iloc[movie_indices][['titleKr', 'vote_count', 'vote_average', 'openYear']]
#     movies = tmp_pmd.iloc[movie_indices][['titleKr', 'genres', 'synopsis', 'vote_count', 'vote_average', 'openYear']]
    
#     # 투표수와 평점이 있는 영화만 필터링
#     vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
#     vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('float')
    
#     # 전체 평균 평점 계산
#     C = vote_averages.mean()
#     # 상위 60% 투표수 이상의 영화만 필터링
#     m = vote_counts.quantile(0.60)
#     qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull())]
    
#     # 투표수를 정수로 변환
#     qualified['vote_count'] = qualified['vote_count'].astype('int')
    
#     # 각 영화의 가중평점 계산
#     qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    
#     # 가중평점 기준으로 상위 10개 영화 추천
#     qualified = qualified.sort_values('wr', ascending=False).head(10)
#     # print(f"추천 영화 (가중평점 기준 상위 10개): {qualified}")
    
#     return qualified

In [ ]:
# improved_recommendations('범죄도시').head(10)

추천할 영화: 범죄도시
2004년 서울. 하얼빈에서 넘어와 단숨에 기존 조직들을 장악하고 가장 강력한 세력인 춘식이파 보스 황사장까지 위협하며 도시 일대의 최강자로 급부상한 신흥범죄조직의 악랄한 보스 장첸. 대한민국을 뒤흔든 장첸 일당을 잡기 위해  오직 주먹 한방으로 도시의 평화를 유지해 온 괴물형사 마석도와  인간미 넘치는 든든한 리더 전일만 반장이 이끄는 강력반은 나쁜 놈들을 한방에 쓸어버릴 끝.짱.나.는. 작전을 세우는데…
영화 인덱스: 2281
추천할 영화들 인덱스: [926, 2939, 2436, 2758, 4350, 780, 351, 2003, 3517, 3564, 3757, 3893, 3442, 3182, 1461, 3843, 1619, 4010, 1112, 1490, 3449, 1137, 924, 2850, 925]


,titleKr,genres,synopsis,vote_count,vote_average,openYear,wr
9799,분노의 질주,"[범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션]",최근 일어난 트럭 도난 사고의 용의자로 LA 폭주족 리더 도미닉이 지목된다. 경찰 ...,9964,6.985,2001,6.675499
581528,악인전,"[범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션]",우연히 연쇄살인마의 표적이 되었다 살아난 조직 보스 장동수와 범인잡기에 혈안이 된 ...,1139,7.800,2019,6.480985
938008,더 킬러: 죽어도 되는 아이,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]",은퇴 후 성공적인 재테크로 호화롭게 생활하는 전설의 킬러 의강이 제멋대로 행동하는 ...,367,7.721,2022,6.357741
955555,범죄도시 3,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]","베트남 납치 살해범 검거 후 7년 뒤, 마석도는 새로운 팀원들과 함께 살인사건을 조...",427,7.200,2023,6.341303
619803,범죄도시 2,"[범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션]","가리봉동 소탕작전 후 4년 뒤, 금천서 강력반은 베트남으로 도주한 용의자를 인도받아...",303,7.233,2022,6.329863
110410,마약전쟁,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]",중국 본토의 수사관 장(손홍뢰)은 어느 날 홍콩 마약 조직의 보스 차이(고천락)를 ...,268,6.900,2014,6.315474
331576,살파랑2: 운명의 시간,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]","형사 ‘진지걸’(오경)은 마약중독자로 신분을 속여 장기밀매조직에 잠입하지만, 계획이...",244,6.900,2016,6.313798
513692,레이징 파이어,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]",강력 범죄 수사대에서 함께 믿고 일하던 베테랑 경찰 장충방과 그의 후배 추강아오. ...,216,6.928,2022,6.312557
42120,용호풍운,"[범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션, 범죄 액션]","비밀경찰의 희생으로 경찰이 딜레마에 빠진 가운데, 사건을 맡은 수사관 라우는 비밀경...",140,7.000,1987,6.308120
706972,나르코 서브,"[액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄, 액션 범죄]",마약단속국에서 인정받는 요원으로 누구보다 가족을 사랑하는 스트라이커. 여대생 납치 ...,259,6.658,2021,6.307376


## 제목 + 장르 + 감독 + 배우 검색 결과 

먼저 제목 + 장르 + 감독 + 배우 를 합친 feature을 하나 만들자.

In [60]:
# smd에 md를 복사해서 넣는다.
smd = md.copy()

In [61]:
# 감독의 가중치를 3배로 넣기 위해 가중치를 3번 넣는다.
smd['staff'] = pmd['staff'].apply(lambda x: [x,x])
smd['actor'] = pmd['actor'].apply(lambda x: [x,x])

In [62]:
smd['actor'].head()

11           [마크 해밀 해리슨 포드  캐리 피셔, 마크 해밀 해리슨 포드  캐리 피셔]
12    [앨버트 브룩스  엘런 드제너러스  알렉산더 굴드 윌렘 대포 브래드 거렛 앨리슨 제...
22                         [조니 뎁  제프리 러쉬, 조니 뎁  제프리 러쉬]
24                                       [우마 서먼, 우마 서먼]
28           [말론 브란도 로버트 듀발 마틴 쉰 , 말론 브란도 로버트 듀발 마틴 쉰 ]
Name: actor, dtype: object

In [63]:
# 데이터 정리: 리스트 내부의 모든 요소를 문자열로 변환
def prepare_text_column(column):
    def process_element(x):
        # 2차원 리스트인 경우 각 서브 리스트를 문자열로 변환 후, 각 서브 리스트를 공백으로 구분하여 join
        if isinstance(x, list):
            if all(isinstance(i, list) for i in x):  # 2차원 리스트인 경우
                return ' '.join([' '.join(map(str, sub_list)) for sub_list in x])
            return ' '.join(map(str, x))  # 1차원 리스트인 경우
        # 리스트가 아닌 경우 문자열로 변환
        return str(x)
    return column.apply(process_element)

# smd['soup_search'] 컬럼 생성
smd['soup_search'] = (
    prepare_text_column(smd['titleKr']) + ' ' +
    prepare_text_column(smd['titleEn']) + ' ' +
    prepare_text_column(smd['staff']) + ' ' +
    prepare_text_column(smd['actor']) + ' ' +
    prepare_text_column(smd['genres'])
)

In [64]:
smd['soup_search'].head()

11    스타워즈 에피소드 4 - 새로운 희망 Star Wars : Episode Iv - ...
12    니모를 찾아서 Finding Nemo 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤...
22    캐리비안의 해적 - 블랙 펄의 저주 Pirates of the Caribbean: ...
24    킬 빌: 1부 Kill Bill: Vol. 1 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔...
28    지옥의 묵시록 Apocalypse Now 프란시스 포드 코폴라 프란시스 포드 코폴라...
Name: soup_search, dtype: object

In [65]:
md['soup'] = pmd['soup']
md['soup_search'] = smd['soup_search']

In [ ]:
md.head(3)

,kino_id,titleKr,titleEn,titleOri,synopsis,genres,productionYear,posterImage,age_rating,openYear,...,staff,actor,tmdb_id,tmdb_poster_img,tmdb_release_date,vote_count,vote_average,popularity,soup,soup_search
11,42481,스타워즈 에피소드 4 - 새로운 희망,Star Wars : Episode Iv - A New Hope,Star Wars : Episode Iv - A New Hope,"고아로 성장한 루크 스카이워커는 삼촌 아저씨, 아주머니와 함께 혹성 타투이에 살고 ...",SF 어드벤처(모험) 액션 판타지,1977,https://file.kinolights.com/original/content_p...,ALL,1977,...,조지 루카스,마크 해밀 해리슨 포드 캐리 피셔,11,https://image.tmdb.org/t/p/original/7XFfURIFCJ...,1977-05-25,20609.0,8.200,99.770,조지 루카스 조지 루카스 마크 해밀 해리슨 포드 캐리 피셔 SF 어드벤처(모험) ...,스타워즈 에피소드 4 - 새로운 희망 Star Wars : Episode Iv - ...
12,32735,니모를 찾아서,Finding Nemo,Finding Nemo,호기심 가득한 아기 물고기 니모가 인간에게 납치되자 아들바보 아빠 말린은 상상초월 ...,애니메이션 어드벤처(모험) 가족 코미디,2003,https://file.kinolights.com/original/content_p...,ALL,2003,...,앤드류 스탠튼,앨버트 브룩스 엘런 드제너러스 알렉산더 굴드 윌렘 대포 브래드 거렛 앨리슨 제니...,12,https://image.tmdb.org/t/p/original/9ViCYfZ0wh...,2003-05-30,19238.0,7.800,98.186,앤드류 스탠튼 앤드류 스탠튼 앨버트 브룩스 엘런 드제너러스 알렉산더 굴드 윌렘 ...,니모를 찾아서 Finding Nemo 앤드류 스탠튼 앤드류 스탠튼 앤드류 스탠튼 앤...
22,44900,캐리비안의 해적 - 블랙 펄의 저주,Pirates of the Caribbean: The Curse of the Bla...,Pirates of the Caribbean: The Curse of the Bla...,매력 넘치는 해적 캡틴 잭 스패로우에게 수정처럼 맑고 투명한 카리브 해는 어드벤처와...,어드벤처(모험) 액션 코미디 판타지,2003,https://file.kinolights.com/original/content_p...,OVER12,2003,...,고어 버빈스키,조니 뎁 제프리 러쉬,22,https://image.tmdb.org/t/p/original/3ovejwQO4f...,2003-07-09,20612.0,7.800,136.419,고어 버빈스키 고어 버빈스키 조니 뎁 제프리 러쉬 어드벤처(모험) 액션 코미디 판...,캐리비안의 해적 - 블랙 펄의 저주 Pirates of the Caribbean: ...
24,35264,킬 빌: 1부,Kill Bill: Vol. 1,Kill Bill: Vol. 1,"어느 한적한 오후, 행복한 결혼식을 앞둔 더 브라이드와 그녀의 신랑, 그리고 모든 ...",액션 스릴러 범죄 드라마,2003,https://file.kinolights.com/original/content_p...,ADULT,2003,...,쿠엔틴 타란티노,우마 서먼,24,https://image.tmdb.org/t/p/original/lH82lk837f...,2003-10-10,17381.0,8.000,51.092,쿠엔틴 타란티노 쿠엔틴 타란티노 우마 서먼 액션 스릴러 범죄 드라마 액션 스릴러 범...,킬 빌: 1부 Kill Bill: Vol. 1 쿠엔틴 타란티노 쿠엔틴 타란티노 쿠엔...
28,11269,지옥의 묵시록,Apocalypse Now,Apocalypse Now,미 특수부대 윌라드 대위는 커츠 대령을 암살하라는 특명을 받는다. 그는 4명의 병사...,전쟁 드라마 액션,1979,https://file.kinolights.com/original/content_p...,ADULT,1979,...,프란시스 포드 코폴라,말론 브란도 로버트 듀발 마틴 쉰,28,https://image.tmdb.org/t/p/original/giP5yeJBrs...,1979-08-15,8218.0,8.276,45.212,프란시스 포드 코폴라 프란시스 포드 코폴라 말론 브란도 로버트 듀발 마틴 쉰 전쟁...,지옥의 묵시록 Apocalypse Now 프란시스 포드 코폴라 프란시스 포드 코폴라...


In [ ]:
md.to_pickle('final/data/md.pkl')

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity
# import pandas as pd

# def search_recommendations(search_query, smd, top_n=10):
#     """
#     주어진 search_query에 대해 가장 유사한 영화 정보를 반환하는 함수
    
#     Parameters:
#         search_query (str): 검색할 텍스트 (예: "영화 감독 배우")
#         smd (pd.DataFrame): 영화 데이터가 담긴 DataFrame
#         top_n (int): 반환할 결과의 수 (기본값: 10)
    
#     Returns:
#         pd.DataFrame: 유사도 높은 순서대로 상위 N개 결과를 담은 DataFrame
#     """
#     # 한국어 불용어 리스트 (예시)
#     stop_words_ko = ['이', '그', '저', '것', '들', '과', '에', '의', '는', '가', '이랑', '하고', '도', '에서', '을', '를']

#     # TF-IDF 벡터화
#     tfidf = TfidfVectorizer(stop_words=stop_words_ko)

#     # soup_search 컬럼과 search_query를 벡터화
#     smd['soup_search_tfidf'] = smd['soup_search'].apply(lambda x: str(x))  # soup_search가 문자열로 처리되도록 보장
#     smd_tfidf_matrix = tfidf.fit_transform(smd['soup_search_tfidf'])
#     search_query_tfidf = tfidf.transform([search_query])

#     # 코사인 유사도 계산
#     cosine_similarities = cosine_similarity(search_query_tfidf, smd_tfidf_matrix).flatten()

#     # 유사도가 높은 순서대로 인덱스를 정렬
#     related_docs_indices = cosine_similarities.argsort()[::-1]

#     # 상위 N개의 결과를 반환
#     top_results = smd.iloc[related_docs_indices[:top_n]]

#     return top_results['tmdb_id']

In [ ]:
# # 함수 사용 예시
# search_query = "마동석 범죄도시"
# top_n = 10
# top_results = search_recommendations(search_query, smd, top_n)

# md.loc[top_results.apply(str), ['titleKr', 'genres', 'staff', 'actor']].head(10)

,titleKr,genres,staff,actor
479718,범죄도시,액션 범죄,강윤성,마동석 윤계상
619803,범죄도시 2,범죄 액션,이상용,마동석 손석구 최귀화
955555,범죄도시 3,액션 범죄,이상용,마동석 이준혁 아오키 무네타카
428639,두 남자,범죄 액션,이성태,마동석 최민호
86001,통증,드라마 멜로/로맨스,곽경택,권상우 정려원 마동석
520358,챔피언,드라마,김용완,마동석 권율 한예리
581528,악인전,범죄 액션,이원태,마동석 김무열 김성규
540384,원더풀 고스트,코미디 범죄 드라마 멜로/로맨스 판타지,조원희,마동석 김영광 이유영
1017163,범죄도시4,액션 범죄 코미디,허명행,마동석 김무열 박지환 이동휘
933131,황야,액션,허명행,마동석 이희준 이준영 노정의


# 협업 필터링
**비슷한 취향을 가진 사용자들이 좋아한 영화 추천**

surprise 라이브러리 사용
1. SVD를 이용해서 사용자-영화 평점 행렬을 분해하여 숨겨진 패턴을 찾아낸다.
2. 여기서 사용자-영화 평점 행렬이란, 사용자가 영화 A와 B를 평가한 점수를 알고 있을 때, 영화 C를 평가한다면 얼마의 평가를 할 지를 예측할 수 있도록 한다.
   ![](https://i.imgur.com/uxOmEny.png)


이런 식으로 협업 필터링을 이용해서 유저마다 다른 추천을 할 수 있도록 만든다.

In [ ]:
# # surprise 라이브러리의 Reader
# reader = Reader()

In [ ]:
# links = pd.read_csv('movielens-data/ml-latest/links.csv')
# links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
# # NaN 값을 제거한 후 tmdbId를 int 타입으로 변환
# links = links.dropna(subset=['tmdbId'])
# links['tmdbId'] = links['tmdbId'].astype('int')

# links.head()

,movieId,imdbId,tmdbId
0,1,114709,862
1,2,113497,8844
2,3,113228,15602
3,4,114885,31357
4,5,113041,11862


In [ ]:
# not_found_cnt = 0
# for row in md.itertuples():
#     try:
#         md.loc[row.Index, 'movielens_id'] = links.loc[row.tmdb_id]['movieId']
#     except KeyError:
#         not_found_cnt = not_found_cnt+1
# print(not_found_cnt)

3417


In [ ]:
# links.to_pickle('links.pkl')

In [ ]:
# import pandas as pd

# # ratings.dat 파일 경로
# file_path = 'movielens-data/ml-1M/ratings.dat'

# # pandas를 사용하여 파일 읽기 (구분자는 '::'로 설정)
# ratings = pd.read_csv(file_path, sep='::', engine='python', names=['userId', 'movieId', 'rating', 'timestamp'])

# # 데이터 확인
# print(ratings.head())

   userId  movieId  rating  timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291


In [ ]:
# ratings.to_csv('ratings.csv', index=False)

In [ ]:
# import pandas as pd

# # ratings.csv 파일 경로
# file_path = 'movielens-data/ml-1m/ratings.csv'

# # pandas를 사용하여 파일 읽기
# ratings = pd.read_csv(file_path)

# # 데이터 확인
# print(ratings.head())

FileNotFoundError: [Errno 2] No such file or directory: 'movielens-data/ml-1m/ratings.csv'

In [ ]:
# # 'ratings'와 'links' 데이터프레임을 'movieId' 기준으로 병합
# merged_df = ratings.merge(links, on='movieId', how='left')

# # NaN 값을 제거한 후 tmdbId를 int 타입으로 변환
# merged_df = merged_df.dropna(subset=['tmdbId'])
# merged_df['tmdbId'] = merged_df['tmdbId'].astype('int')

# # 결과 확인
# merged_df.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId
0,1,2,3.5,1112486027,113497.0,8844
1,1,29,3.5,1112484676,112682.0,902
2,1,32,3.5,1112484819,114746.0,63
3,1,47,3.5,1112484727,114369.0,807
4,1,50,3.5,1112484580,114814.0,629


In [ ]:
# data = Dataset.load_from_df(merged_df[['userId', 'tmdbId','rating']], reader)
# # data.split(n_folds=5)

# trainset = data.build_full_trainset()
# testset = trainset.build_testset()

In [ ]:
# # 이미 파일로 저장 완료
# svd = SVD()
# # evaluate(svd, data, measures=['RMSE', 'MAE'])

# ####### 기존 커널대로 진행하면 오류나서 수정 #######
# svd.fit(trainset)
# predictions = svd.test(testset)
# accuracy.rmse(predictions)

RMSE: 0.6779


0.677925900778656

In [ ]:
# import pickle

# # svd 모델을 svd_model_small.pkl에 저장
# with open('svd_model_20m.pkl', 'wb') as f:
#     pickle.dump(svd, f)

In [ ]:
# # 모델 불러오기
# with open('svd_model_1m.pkl', 'rb') as f:
#     svd_1m = pickle.load(f)

In [ ]:
# # 모델 불러오기
# with open('svd_model.pkl', 'rb') as f:
#     tmp_svd = pickle.load(f)

In [308]:
# # svd 정확도 테스트
# predictions = svd.test(testset)
# accuracy.rmse(predictions)

평균 Root Mean Sqaure Error는 0.6473으로,이 경우에는 충분하다. 이제 데이터 세트를 학습하고 예측해보자.

주어진 결과는 SVD(Singular Value Decomposition) 기반의 추천 시스템에서 특정 사용자와 아이템에 대한 평점 예측 결과를 나타냅니다. 하나씩 살펴보겠습니다.

1. 결과 해석

Prediction(uid=1, iid=302, r_ui=3, est=2.6904498903376326, details={'was_impossible': False})
	•	uid=1
	•	사용자 ID입니다. 여기서는 사용자 1번을 의미합니다.
	•	iid=302
	•	아이템 ID입니다. 여기서는 아이템 302번을 의미합니다. (예: 영화, 책, 제품 등)
	•	r_ui=3
	•	이 사용자가 실제로 평가한 평점(r_ui, rating user-item)입니다.
	•	실제 평점이 존재하는 경우 이 값은 모델이 검증 중 사용합니다.
	•	만약 실제 평점이 없다면, 이 값은 None일 수 있습니다.
	•	est=2.6904498903376326
	•	**예측 평점(estimated rating)**입니다.
	•	사용자 1번이 아이템 302번에 대해 줄 것이라고 SVD 모델이 예측한 평점입니다.
	•	여기서는 약 2.69로 예측되었습니다.
	•	details={'was_impossible': False}
	•	모델이 예측을 성공적으로 수행했는지를 나타냅니다.
	•	was_impossible=False는 예측이 가능했음을 의미합니다.
	•	만약 예측에 필요한 데이터(사용자/아이템 정보)가 부족하면 was_impossible=True가 됩니다.

2. 결과의 의미

이 결과를 통해 사용자 1번이 아이템 302번에 대해 실제로 평가한 평점(r_ui)은 3점이었고, 모델은 이 평점을 약 2.69로 예측했음을 알 수 있습니다.
	•	이 차이(r_ui와 est의 차이)는 모델의 예측 오차를 계산하는 데 사용됩니다.
	•	예를 들어, 여러 평점 예측 결과에 대해 RMSE(Root Mean Square Error)를 계산하면 모델 성능을 평가할 수 있습니다.

3. 활용 방안

이런 예측 결과는 다음과 같은 방식으로 활용됩니다:
	1.	추천 생성:
	•	실제 평점(r_ui)이 없는 아이템에 대해 est 값을 기준으로 상위 추천 목록을 생성할 수 있습니다.
	•	예를 들어, 사용자 1번에게 est 값이 높은 아이템들을 추천합니다.
	2.	모델 성능 평가:
	•	실제 평점(r_ui)과 예측 평점(est)의 차이를 기반으로 모델의 성능을 측정합니다.
	•	대표적인 평가지표:
	•	RMSE (Root Mean Square Error): 예측의 평균 오차를 계산.
	•	MAE (Mean Absolute Error): 절대 오차의 평균 계산.

4. 결론

이 예측은 SVD 모델이 학습한 사용자-아이템 간 관계를 기반으로 생성된 것입니다.
	•	**r_ui=3**과 est=2.69 간 차이를 줄이는 것이 모델 개선의 목표입니다.
	•	details={'was_impossible': False}는 데이터가 충분해 예측이 가능했음을 나타냅니다.

추가적인 질문이나 구현에 대해 궁금한 점이 있으면 말씀해주세요! 😊

# 하이브리드 추천 시스템

In [ ]:
# cosine_sim_description = np.load(
#     'final/data/cosine_sim_description.npy')
# cosine_sim_metadata = np.load(
#     'final/data/cosine_sim_metadata.npy')
# with open('final/data/indices.pkl', 'rb') as f:
#     indices = pickle.load(f)

In [ ]:
# md = pd.read_pickle('final/data/md.pkl')

In [ ]:
# tmp_tmp_pmd = tmp_pmd.copy()

In [ ]:
# """

# 필요한 것
# 1. pmd: soup 있던 md
# 2. svd: 협업 필터링
# 3. cosine_sim_description, cosine_sim_meta
# 4. indices: title → id(tmdb)
# """

# def hybrid(userId, title, weight_description=0.25, weight_metadata=0.75, min_vote_count=1):
#     # 영화 제목을 통해 인덱스 가져오기
#     idx = indices[title]
    
#     # 영화 설명 유사도 (TF-IDF)와 메타데이터 유사도 (감독, 배우, 장르) 가져오기
#     sim_scores_description = list(enumerate(cosine_sim_description[int(idx)]))
#     sim_scores_metadata = list(enumerate(cosine_sim_metadata[int(idx)]))
    
#     # 두 유사도 행렬을 결합 (가중합)
#     sim_scores = [
#         (i[0], weight_description * i[1] + weight_metadata * j[1])
#         for i, j in zip(sim_scores_description, sim_scores_metadata)
#     ]
    
#     # 유사도 순으로 정렬 (가장 유사한 것부터)
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
#     # 상위 25개 영화 선택 (자기 자신 제외)
#     sim_scores = sim_scores[1:26]
#     movie_indices = [i[0] for i in sim_scores]

#     print(movie_indices)
    
#     # 영화 데이터 가져오기 (제목, 투표수, 평균 평점, 년도, id)
#     movies = tmp_pmd.iloc[movie_indices][['titleKr', 'vote_count', 'vote_average', 'openYear', 'tmdb_id']]
    
#     # 최소 vote_count를 기준으로 필터링 (예: 100 이상)
#     movies = movies[movies['vote_count'] >= min_vote_count]

#     # SVD를 통해 예측된 평점 계산
#     movies['est'] = movies['tmdb_id'].apply(lambda x: svd_1m.predict(userId, x).est)
    
#     # 예측된 평점 기준으로 상위 10개 영화 추천
#     movies = movies.sort_values('est', ascending=False)
    
#     return movies.head(10)

In [149]:
# result = search_recommendations('범죄 도시', smd, 10)
# md.loc[top_results.apply(str), ['titleKr', 'genres', 'staff', 'actor']].head(10)

In [ ]:
# hybrid(1, '범죄도시')

[926, 2939, 2436, 2758, 4350, 780, 3564, 351, 3517, 2003, 3893, 3757, 3442, 1461, 3182, 1619, 4010, 3843, 1137, 1112, 3449, 1490, 924, 2850, 925]


,titleKr,vote_count,vote_average,openYear,tmdb_id,est
58414,테러리스트,15.0,6.800,1999,58414,3.740246
267573,하이힐,91.0,6.945,2014,267573,3.740246
602661,추룡 2: 패왕,26.0,5.900,2021,602661,3.740246
58409,특경도룡,35.0,6.400,1989,58409,3.740246
277519,작전명: 제트스톰,44.0,5.900,2014,277519,3.740246
795321,아수라도,2.0,4.500,2021,795321,3.740246
99582,첩혈남아,15.0,5.433,1994,99582,3.740246
110410,마약전쟁,268.0,6.900,2014,110410,3.740246
938008,더 킬러: 죽어도 되는 아이,367.0,7.721,2022,938008,3.740246
331576,살파랑2: 운명의 시간,244.0,6.900,2016,331576,3.740246


In [ ]:
# hybrid(1, '어게인 1997')

,titleKr,vote_count,vote_average,openYear,tmdb_id,est
1079196,봉태리,1.0,5.000,2023,1079196,3.740246
739064,쌍면호,2.0,4.500,2020,739064,3.740246
1127166,눈물을 만드는 사람,775.0,6.519,2024,1127166,3.740246
884892,기문법사: 술사들의 전쟁,1.0,9.000,2021,884892,3.740246
986263,울프킨,18.0,5.800,2023,986263,3.740246
1047223,부르달락: 흡혈귀,28.0,6.000,2023,1047223,3.740246
1072856,생각 그런거 없는데요,3.0,5.700,2022,1072856,3.740246
845084,사랑의 형태,5.0,4.900,2023,845084,3.740246
748167,어글리,623.0,6.000,2024,748167,3.740246
734253,아디푸루시,88.0,5.068,2023,734253,3.740246
